In [1]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.5 MB/s 
     |████████████████████████████████| 16.3 MB 78 kB/s 
     |████████████████████████████████| 73 kB 2.0 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 34.7 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [4]:
%cd drive/MyDrive/TAL/tp_nmt/

/content/drive/MyDrive/TAL/tp_nmt


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Question 1

Ces fichiers permettent d'entrainer le réseau de neuronnes. Il y a ainsi un fichier contenant les phrases en anglais et un autre contenant les traduction en français. Ainsi un premier fichier servira d'input et le deuxième d'output. <span style="color:red">ca veut dire quoi que un fichier "serve d'output"?</span>

## Question 2

In [5]:
%%shell
cd BTEC-en-fr/dev/
awk -F '\' '{print $NF}' IWSLT10.devset1_CSTAR03.en.txt > IWSLT10.devset1_CSTAR03.en.clean.txt
perl ../../tokenizer.perl -l en -lc < ./IWSLT10.devset1_CSTAR03.en.clean.txt > IWSLT10.devset1_CSTAR03.en.tok.txt
awk -F '\' '{print $NF}' IWSLT10.devset1_CSTAR03.fr.txt > IWSLT10.devset1_CSTAR03.fr.clean.txt
perl ../../tokenizer.perl -l en -lc < ./IWSLT10.devset1_CSTAR03.fr.clean.txt > IWSLT10.devset1_CSTAR03.fr.tok.txt
cd ../test/
awk -F '\' '{print $NF}' IWSLT09_BTEC.testset.en.txt > IWSLT09_BTEC.testset.en.clean.txt
perl ../../tokenizer.perl -l en -lc < ./IWSLT09_BTEC.testset.en.clean.txt > IWSLT09_BTEC.testset.en.tok.txt
awk -F '\' '{print $NF}' IWSLT09_BTEC.testset.fr.txt > IWSLT09_BTEC.testset.fr.clean.txt
perl ../../tokenizer.perl -l en -lc < ./IWSLT09_BTEC.testset.fr.clean.txt > IWSLT09_BTEC.testset.fr.tok.txt
cd ../train/
awk -F '\' '{print $NF}' IWSLT10_BTEC.train.en.txt > IWSLT10_BTEC.train.en.clean.txt
perl ../../tokenizer.perl -l en -lc < ./IWSLT10_BTEC.train.en.clean.txt >IWSLT10_BTEC.train.en.tok.txt
awk -F '\' '{print $NF}' IWSLT10_BTEC.train.fr.txt > IWSLT10_BTEC.train.fr.clean.txt
perl ../../tokenizer.perl -l en -lc < ./IWSLT10_BTEC.train.fr.clean.txt >IWSLT10_BTEC.train.fr.tok.txt

Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1


Les fichiers clean permettent d'enlever les identifiant au début de chaque phrase et le fichiers tok permettent de gérer la ponctuation.
L'intérêt est de pouvoir entrainer le réseau en reconnaissant seulement le contenu de chaque phrase et en reconnaissant les ponctuations. Chaque token est alors isolé et on pourra construire notre vocabulaire.

## Question 3

La partie train est utilisé pendant le processus d'apprentissage afin d'ajuster les paramètres...
La partie dev est utilisé pendant le processus de validation afin de régler les hyperparamètres. 
La partie test est utilisé pour évaluer les prédictions du modèle final mais ne participer pas à son entrainement

## Question 4

In [11]:
!onmt_build_vocab -config config-base.yaml -n_sample -1

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 19:35:24,970 INFO] Counter vocab from -1 samples.
[2022-02-27 19:35:24,970 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-27 19:35:24,983 INFO] train's transforms: TransformPipe()
[2022-02-27 19:35:25,289 INFO] Counters src:10413
[2022-02-27 19:35:25,289 INFO] Counters tgt:8194
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 71, in main
    build_vocab_main(opts)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 55, in build_vocab_main
    save_counter(src_counter, opts.src_vocab)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 44, in save_counter
    check_path(save_path, exist_ok=opts.overwrite, log=logger.warning)
  File "/usr/local/lib/python3.7/dist-packages/onmt/utils/misc.py", line 17, in

Le modèle de TA a besoin d'un vocabulaire afin de pouvoir calculer les probabilités de traduction des mots et ajuster ces dernières.
Un vocabulaire basé sur les données d'apprentissage sera plus précis et nous permettra de développer un meilleur modèle.

<span style="color:red">Ca veut dire quoi "plus précis"? Les tokens du vocabulaire basé sur l'ensemble d'apprentissage sont suffisants pour l’entraînement du modèle parce que pendent l’entraînement le modèle ne voit que des phrases issues du training set. En autre, utiliser un vocabulaire plus vaste serait inefficace et gourmand en calcul et en mémoire, car des tokens de ce vocabulaire ne seront jamais utilisés pour l’entraînement.</span> 

## Question 5

Counters src indique le nombre de token du langage source et Counters tgt le nombre de tokens du langage cible <span style="color:red">plus précisement: le nombre de tokens uniques présents dans le vocabulaire source et cible.</span> 

In [ ]:
!onmt_train -config config-base.yaml

[2022-02-27 11:46:02,098 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 11:46:02,433 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 11:46:02,434 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 11:46:02,434 INFO] Parsed 2 corpora from -data.
[2022-02-27 11:46:02,435 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 11:46:02,435 INFO] Loading vocab from text file...
[2022-02-27 11:46:02,435 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 11:46:02,733 INFO] Loaded src vocab has 10413 tokens.
[2022-02-27 11:46:02,738 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 11:46:03,042 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 11:46:03,046 INFO] Building fields with vocab in counters...
[2022-02-27 11:46:03,055 INFO]  * tgt vocab size: 8198.
[2022-02-27 11:46:03,068 INFO]  * src vocab size:

## Question 6

Acc permet de calculer l'accuracy, c'est à dire le pourcentage de token correctement traduit. Il est donc préférable d'avoir la précision la plus élevée possible

La perplexité permet de mesurer si la distribution de probabilité prédit bien un échantillon. Il est préférable d'avoir une faible ppl, ce qui indique que la distribution est bonne

## Question 7

train_step : nombre d'étapes de descente de gradient sur les données d'entrainement <span style="color:red">Nombre total d'étapes d'optimisation des poids du modèle, où chaque étape est constitué d'un passage en avant d'un lot (batch) de données d'apprentissage dans le modèle (forward pass) et d'un passage en arrière des gradients générés par la fonction de perte (backward pass). </span>

valid_step : nombre d'étapes de descente de gradient sur les données de validation <span style="color:red">Il n'y a pas de l'optimization pendent la validation. Elle sert seulement à évaluer le modèle et décider si changer des hyperparametres où arreter l'entrainement.</span> 

enc_layers : nombre de couches de l'encodeur

dec_layers : nombre de couches du décodeur

enc_rnn_size : nombre de couches cachées de l'encodeur <span style="color:red">Taille des representations vectorielles dans l'encodeur.</span>

dec_rnn_size : nombre de couche cachées du décodeur <span style="color:red">Taille des représentations vectorielles dans le décodeur.</span>

batch_size : Taille d'un groupe de données <span style="color:red">C'est à dire? batch_size=nombre d'exemples d'entrainement (phrase source + phrase de réference) passé au modèle à chaque étape d'entrainement (avant le calcul du gradient)</span>

## Question 8

In [12]:
%%shell
onmt_translate -model models/base/model_step_1250.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred1250.txt
onmt_translate -model models/base/model_step_1875.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred1875.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000.txt
onmt_translate -model models/base/model_step_625.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred625.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 19:40:38,279 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 19:40:42,607 INFO] PRED AVG SCORE: -1.1741, PRED PPL: 3.2352
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

L'algorithme BLEU utilisé dans l'évaluation de la performance de la traduction produit un score entre 0 et 1. Une valeur proche de 1 correspond à une très bonne qualité de traduction tandis qu'une valeur proche de 0 correspond à une qualité médiocre de traduction. Ainsi, l'idéal serait d'obtenir un score de 1, même si en pratique, on cherche simplement à maximiser le score BLEU et à être aux environs de la valeur 1. <span style="color:red">Ca veut dire quoi "bonne qualité de traduction"? En fait, un meilleur BLEU score ne veut dire pas **toujours** une meilleure traduction</span>

# Question 9

In [14]:
%%shell
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred1250.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred1875.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred625.txt

BLEU = 13.47, 48.7/20.3/11.9/4.7 (BP=0.880, ratio=0.887, hyp_len=3170, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 16.82, 47.6/21.0/12.7/6.3 (BP=1.000, ratio=1.030, hyp_len=3681, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 18.32, 52.0/24.3/14.8/7.4 (BP=0.952, ratio=0.953, h

Avec les résultats précédents, on retient le modèle qui a le score BLEU le plus proche de 1, il s'agit du modèle model_step_2000

# Question 10

In [18]:
%%shell
onmt_train -config Question10/config-base.yaml

[2022-02-27 20:10:21,839 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 20:10:21,840 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 20:10:21,841 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 20:10:21,841 INFO] Parsed 2 corpora from -data.
[2022-02-27 20:10:21,842 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 20:10:21,842 INFO] Loading vocab from text file...
[2022-02-27 20:10:21,843 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 20:10:21,871 INFO] Loaded src vocab has 10413 tokens.
[2022-02-27 20:10:21,877 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 20:10:21,899 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 20:10:21,903 INFO] Building fields with vocab in counters...
[2022-02-27 20:10:21,913 INFO]  * tgt vocab size: 8198.
[2022-02-27 20:10:21,929 INFO]  * src vocab size:

In [20]:
%%shell
onmt_translate -model Question10/model_step_3000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred3000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred3000.txt
onmt_translate -model Question10/model_step_4000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred4000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred4000.txt
onmt_translate -model Question10/model_step_5000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred5000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred5000.txt
onmt_translate -model Question10/model_step_6000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred6000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred6000.txt
onmt_translate -model Question10/model_step_7000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred7000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred7000.txt
onmt_translate -model Question10/model_step_8000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred8000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred8000.txt
onmt_translate -model Question10/model_step_9000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred9000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred9000.txt
onmt_translate -model Question10/model_step_10000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question10/pred10000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question10/pred10000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 20:18:40,590 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 20:18:45,174 INFO] PRED AVG SCORE: -0.8394, PRED PPL: 2.3151
BLEU = 23.89, 56.7/29.5/19.7/11.9 (BP=0.955, ratio=0.956, hyp_len=3417, ref_len=3574)
It is 

In [23]:
%%shell
onmt_train -config Question10/config-base.yaml --early_stopping 1

[2022-02-27 20:25:41,908 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 20:25:41,911 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 20:25:41,911 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 20:25:41,916 INFO] Parsed 2 corpora from -data.
[2022-02-27 20:25:41,917 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 20:25:41,917 INFO] Loading vocab from text file...
[2022-02-27 20:25:41,917 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 20:25:41,946 INFO] Loaded src vocab has 10413 tokens.
[2022-02-27 20:25:41,952 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 20:25:41,972 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 20:25:41,976 INFO] Building fields with vocab in counters...
[2022-02-27 20:25:41,986 INFO]  * tgt vocab size: 8198.
[2022-02-27 20:25:42,000 INFO]  * src vocab size:

Augmenter le nombre de training steps permet de renforcer l'apprentissage et souvent d'augmenter le score BLEU. Cependant, il est possible d'atteindre un stade où augmenter le nombre de training steps est sans influence voire avec une influence négative sur la performance BLEU. A 7000 steps, le score bleu rediminue et réaugmente à 8000 steps avant de rediminuer
 On peut utiliser une méthode d'early stopping afin de déterminer le nombre idéal de training steps avec l'option early7.2.3_stopping. En effet, cette option permet de s'arrêter si la performance n'a pas augmenté au bout du nombre spécifié de training steps et donc de déterminer le nombre idéal de training steps selon les besoins.

## Question 11

In [26]:
%%shell
onmt_train -config Question11/config-base2.yaml

[2022-02-27 20:36:31,933 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 20:36:31,934 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 20:36:31,935 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 20:36:31,935 INFO] Parsed 2 corpora from -data.
[2022-02-27 20:36:31,936 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 20:36:31,936 INFO] Loading vocab from text file...
[2022-02-27 20:36:31,937 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 20:36:31,965 INFO] Loaded src vocab has 10413 tokens.
[2022-02-27 20:36:31,971 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 20:36:31,991 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 20:36:31,995 INFO] Building fields with vocab in counters...
[2022-02-27 20:36:32,006 INFO]  * tgt vocab size: 8198.
[2022-02-27 20:36:32,020 INFO]  * src vocab size:

In [27]:
%%shell
onmt_translate -model Question11/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question11/pred2000.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question11/pred2000.txt

[2022-02-27 20:38:48,683 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 20:38:53,322 INFO] PRED AVG SCORE: -1.0319, PRED PPL: 2.8063
BLEU = 15.65, 52.0/23.1/13.8/6.5 (BP=0.863, ratio=0.872, hyp_len=3116, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores fro

In [28]:
%%shell
onmt_train -config Question11/config-base3.yaml

[2022-02-27 20:39:07,684 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 20:39:07,686 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 20:39:07,686 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 20:39:07,687 INFO] Parsed 2 corpora from -data.
[2022-02-27 20:39:07,688 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 20:39:07,688 INFO] Loading vocab from text file...
[2022-02-27 20:39:07,688 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 20:39:07,719 INFO] Loaded src vocab has 10413 tokens.
[2022-02-27 20:39:07,724 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 20:39:07,744 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 20:39:07,749 INFO] Building fields with vocab in counters...
[2022-02-27 20:39:07,759 INFO]  * tgt vocab size: 8198.
[2022-02-27 20:39:07,773 INFO]  * src vocab size:

In [29]:
%%shell
onmt_translate -model Question11/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question11/pred2000_3.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question11/pred2000_3.txt

[2022-02-27 20:42:04,216 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 20:42:10,136 INFO] PRED AVG SCORE: -1.2179, PRED PPL: 3.3802
BLEU = 10.47, 42.4/14.9/8.3/2.8 (BP=0.951, ratio=0.952, hyp_len=3402, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from

Avec 1 couche pour 2000 steps, on avait un score bleu de 18,32%
Pour 2 couches, on a une score de 15,65 % 
Pour 3 couches, on a un score de 10,47 %
Cela ne permet donc pas toujours d'obtenir des meilleurs résultats car il y a plus de poids à mettre à jour et pas toujours assez d'étapes d'entrainement

<span style="color:red">C'esut une bonne observation. Toutefois Il aurait fallu utiliser l'option early_stopping pour bien repondre à cette question. En effet, si l'on utilise pas cette option, on risque de comparer des modèles qui sont sous-entrainé (c'est votre cas) ou sur-entrainés (overfitted),  sans donc pouvoir en tirer des conclusions. Vous aurez pu aussi essayer d'autres combinaisons de couches (e.g. 1+3,2+1,3+3...) pour pouvoir tirer des conclusions plus solides.</span>

## Question 12

In [31]:
%%shell
onmt_train -config Question12/config-base384.yaml

[2022-02-27 20:47:43,564 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 20:47:43,568 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 20:47:43,568 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 20:47:43,568 INFO] Parsed 2 corpora from -data.
[2022-02-27 20:47:43,569 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 20:47:43,570 INFO] Loading vocab from text file...
[2022-02-27 20:47:43,570 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 20:47:43,597 INFO] Loaded src vocab has 10413 tokens.
[2022-02-27 20:47:43,603 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 20:47:43,622 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 20:47:43,627 INFO] Building fields with vocab in counters...
[2022-02-27 20:47:43,637 INFO]  * tgt vocab size: 8198.
[2022-02-27 20:47:43,652 INFO]  * src vocab size:

In [32]:
%%shell
onmt_translate -model Question12/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question12/pred2000_384.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question12/pred2000_384.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 20:49:31,683 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 20:49:37,874 INFO] PRED AVG SCORE: -0.9189, PRED PPL: 2.5066
BLEU = 20.05, 55.3/26.8/17.1/9.3 (BP=0.910, ratio=0.914, hyp_len=3267, ref_len=3574)
It is n

In [33]:
%%shell
onmt_train -config Question12/config-base384.yaml

[2022-02-27 20:49:44,968 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 20:49:44,969 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 20:49:44,970 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 20:49:44,970 INFO] Parsed 2 corpora from -data.
[2022-02-27 20:49:44,971 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 20:49:44,971 INFO] Loading vocab from text file...
[2022-02-27 20:49:44,972 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 20:49:45,001 INFO] Loaded src vocab has 10413 tokens.
[2022-02-27 20:49:45,010 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 20:49:45,032 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 20:49:45,036 INFO] Building fields with vocab in counters...
[2022-02-27 20:49:45,046 INFO]  * tgt vocab size: 8198.
[2022-02-27 20:49:45,059 INFO]  * src vocab size:

In [34]:
%%shell
onmt_translate -model Question12/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Question12/pred2000_512.txt
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Question12/pred2000_512.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 20:51:33,776 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 20:51:40,137 INFO] PRED AVG SCORE: -0.9113, PRED PPL: 2.4876
BLEU = 18.96, 54.8/26.2/16.1/8.4 (BP=0.904, ratio=0.908, hyp_len=3246, ref_len=3574)
It is n

Avec 256 unités, on a 18,32% pour 2000 étapes.
Avec 384 unités, on a 20,05 %
Avec 512 unités, on a 18,96%
On obtient ainsi des meilleurs résultats.

<span style="color:red">Pourquoi? Sinon, vous auraît toujours du utiliser le early stopping (voir commentaire précédente).</span>

# Question 14


Le Beam Search est une technique qui permet de rechecher plusieurs candidats à la fois au lieu d'un seul, supposé être le meilleur, dans le cadre de la traduction pour chaque séquence d'entrée à chaque timestep. La méthode repose sur des probabilités conditionnelles et permet de retenir les B meilleures alternatives (celles qui ont la probabilité la plus élevée) où B est le paramètre de Beam Search. La taille par défaut du Beam Search est de 1 (un seul candidat proposé).

<span style="color:red"> La Beam Search est un algo qu'on utilise au décodage pour chercher la chaine des tokens cible qui mieux correspond à la source.  Cet algorithme est une approximation efficiente de l'algorithme de parcours en largeur. Pour un beam search initialisé à **B**, le modèle devra sélectionner les **B** séquences les plus susceptibles d'apparaître à chaque étape du décodage (étape de décodage=décodage du token suivant) en utilisant les probabilités d'occurence. Théoriquement, la traduction faite par le modèle serait d'autant meilleure que la taille du beam search est grande, car la recherche sera plus aple. Toutefois, un beam search très grand nécessitera d'avantage de ressources (mémoires et processeurs). </span>

## Question 15

In [40]:
%%shell
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam1.txt --beam_size 1
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam1.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam2.txt --beam_size 2
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam2.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam3.txt --beam_size 3
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam3.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam4.txt --beam_size 4
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam4.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam5.txt --beam_size 5
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam5.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam6.txt --beam_size 6
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam6.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam7.txt --beam_size 7
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam7.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam8.txt --beam_size 8
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam8.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam9.txt --beam_size 9
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam9.txt
onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output models/base/pred2000_beam10.txt --beam_size 10
perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000_beam10.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 21:02:41,332 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 21:02:43,071 INFO] PRED AVG SCORE: -1.1794, PRED PPL: 3.2523
BLEU = 16.98, 47.7/21.2/12.7/6.5 (BP=1.000, ratio=1.077, hyp_len=3849, ref_len=3574)
It is n

Le score est maximum avec une beam de taille 4

<span style="color:red"> Comment évolue le score BLEU ? Pourquoi? </span>